In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

from PIL import Image, ImageFilter
import PIL
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle

In [2]:
from gen_pic_utils import *
import populate_library

## Select car and road type

In [9]:
car_types = [9]
road_type = 57

# Initialize car configurations (x,y,scale)
configurations = []
for _ in car_types:
    configurations.append((0,0,1))
    

In [10]:
# Slider parameters
car_widget = FloatSlider(min=0, max=len(car_types)-1, step=1, value=0)
x_widget = FloatSlider(min=1, max=1242, step=1)
y_widget = FloatSlider(min=1, max=375, step=1)
scale_widget = FloatSlider(min=0.1, max=2.0, step=0.1, value=1)

def update_car_conf_widgets(*args):
    (x,y,scale) = configurations[int(car_widget.value)]
    x_widget.value = x
    y_widget.value = y
    scale_widget.value = scale
    
car_widget.observe(update_car_conf_widgets, 'value')

def update_image(car,x,y,scale):
    configurations[int(car_widget.value)] = (x_widget.value,y_widget.value,scale_widget.value)
    
    lib = populate_library.populateLibrary()     
        
    cars = []
    for car_type in car_types:
        cars.append(lib.getElement("cars", car_type))
        
    road = lib.getElement("roads", 57)    
    
    for car, (x,y,scale) in zip(cars,configurations):
        car.data = car.data.resize((int(car.data.size[0]*scale), int(car.data.size[1]*scale)), PIL.Image.ANTIALIAS)
        car_mask = car.data.getdata(3)
        mask = Image.new('L', car.data.size, color = 0)
        mask.putdata(car_mask)        
        road.data.paste(car.data, (int(x),int(y)), mask)
        
    display.display(road.data)


In [11]:
# Start GUI
interact(update_image,car=car_widget, x=x_widget, y=y_widget, scale=scale_widget);


In [9]:
configurations

[(574.0, 109.0, 0.5), (631.0, 118.0, 1.0), (369.0, 130.0, 1.0)]

In [37]:
image_configurations = load('image_configurations')
image_configurations[road_type] += configurations
save('image_configurations',image_configurations)

In [8]:
def save(filename,data):
    with open(filename, 'w+') as f:
        pickle.dump(data, f)
        
def load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)